draft/Not done and ive gotta check my maths


accuracy not tested, probably better that way

In [ ]:
embedder = EGNN1(dim=3, depth, max_nodes=150)
nn_pk=EGNN_Network(dim=2, depth, max_nodes=300)

net = embedder + egnn_pk + MHAtt 
optimizer=optimizer
lossfn = lossfn

def EGNNEmbedder(feats, coors, nloops):
    """TODO shuffle OR math-permequivariance check or figure out permutation matrices"""

    for n in range(nloops):
        out = embedder(feats, coors) 
    return out, coors

def ProteinAtt(rep, coors):
    """aggregates and then does system wide self-attention . combines positional information and math operated latent space rep (which also holds positional info)
    
    TODO remember the bacwards propagate error and achtung with detaching or just find torch laplace/dots and more thoroughly check mathlegitness
    
    to do deal with detaching and lists/np
    
    have only tested that it holds its signals from its higher dimenensions, but not if it makes good predictions
    
    if preds bad and log of signal is mathlegal, it is free coarse graining / reduce num dims and has same effect at this resolution"""

    dots=[]
    for x in reps.values(): #resi-species level resolution
        z=np.sum(np.sum(np.gradient(np.gradient(x.detach().numpy())), axis=-1)[0],axis=-0)
        r=norms[species]
        o =np.vstack([[z],[r]])
        dots.append(np.dot(o[0,:]-np.mean(o[0,:]),o[1,:]-np.mean(o[1,:])))
    return dots


def get_spheres(all_coords, site, cutoff_radius):
    nbrs = NearestNeighbors(cutoff_radius).fit(all_coords) #prob np
    _, indices = nbrs.radius_neighbors(site) #list or np prob
    return all_coords[indices]
        
def Run(nepochs):
    for i in nepochs:
        for j in range(emb_layers):

        ########################solute-solvent############################################label-independent############################################
            for resi in system:
                feats, coors = EGNNEmbedder(dim=3, depth=depth)([species, coors], nloops=1) 
                updated_coors[resi] = coors
                reps[resi] = AttAgg(feats, coors)
        
        all_feats = ProteinAtt(reps) #[real_number_resis, 1]


        ##########################solution########################################need-labels##########################################################
        hoods = [(sample, targets[sample], get_spheres(updated_coors.values(), site_indices[sample], cutoff_radius) for sample in labeled_samples)] #[labeled_resis, 1]


        for sample in hoods: 
            feats, coors = egnn_pk(feats, coors)
            feats = MHAtt(feats)
            reps[resi] = AttAgg(feats, coors)
            
            out = ProteinAtt(all_feats) 

            pks = target_function(out) #maybe optional but probably not
            #optional activate: softmax, tanh 

        #####################################

            loss = lossfn(pks, targets) #protein wide if so short on time, or try to capture sequence by vutting it up. i like batch size 1 tho
            loss.backward()

            titratable_pks = ProteinAtt(all_feats) #[labeled_resis, 1]




